In [ ]:
import pandas as pd
import numpy as np

In [ ]:
columns={
    '3.feed_1.SO2_PPM': 'SO2_PPM',
    '3.feed_1.H2S_PPM': 'H2S_PPM',
    '3.feed_1.SIGTHETA_DEG': 'SIGTHETA_DEG',
    '3.feed_1.SONICWD_DEG': 'SONICWD_DEG@',
    '3.feed_1.SONICWS_MPH': 'SONICWS_MPH',
    '3.feed_11067.CO_PPB..3.feed_43.CO_PPB': 'CO_PPB',
    '3.feed_11067.NO2_PPB..3.feed_43.NO2_PPB': 'NO2_PPB',
    '3.feed_11067.NOX_PPB..3.feed_43.NOX_PPB': 'NOX_PPB',
    '3.feed_11067.NO_PPB..3.feed_43.NO_PPB': 'NO_PPB',
    '3.feed_11067.PM25T_UG_M3..3.feed_43.PM25T_UG_M3': 'PM2_5_',
    '3.feed_11067.SIGTHETA_DEG..3.feed_43.SIGTHETA_DEG': 'SIGTHETA_DEG',
    '3.feed_11067.SONICWD_DEG..3.feed_43.SONICWD_DEG': 'SONICWD_DEG@',
    '3.feed_11067.SONICWS_MPH..3.feed_43.SONICWS_MPH': 'SONICWS_MPH',
    '3.feed_29.PM10_UG_M3': 'PM10_',
    '3.feed_3506.PM2_5': 'PM2_5_1',
    '3.feed_3506.OZONE': 'OZONE',
    '3.feed_24.PM10_UG_M3': 'PM_10',
    '3.feed_3508.PM2_5': 'PM_2_5_2',
    '3.feed_3.PM10B_UG_M3..3.feed_3.PM10_640_UG_M3': 'PM10_1',
    '3.feed_23.CO_PPM..3.feed_23.CO_PPB': 'CO_PPM_CO_PPB',
    '3.feed_28.H2S_PPM': 'H2S_PPM',
    '3.feed_28.SO2_PPM': 'SO2_PPM',
    '3.feed_28.SIGTHETA_DEG': 'SIGTHETA_DEG',
    '3.feed_28.SONICWD_DEG': 'SONICWD_DEG@',
    '3.feed_28.SONICWS_MPH': 'SONICWS_MPH',
    '3.feed_1.PM25B_UG_M3..3.feed_1.PM25T_UG_M3..3.feed_1.PM25_640_UG_M3': 'PM2_5_3',
    '3.feed_26.OZONE_PPM': 'OZONE_PPM',
    '3.feed_26.SONICWS_MPH': 'SONICWS_MPH',
    '3.feed_26.SONICWD_DEG': 'SONICWD_DEG@',
    '3.feed_26.SIGTHETA_DEG': 'SIGTHETA_DEG',
    '3.feed_3.SO2_PPM': 'SO2_PPM',
    '3.feed_3.SONICWD_DEG': 'SONICWD_DEG@',
    '3.feed_3.SONICWS_MPH': 'SONICWS_MPH',
    '3.feed_3.SIGTHETA_DEG': 'SIGTHETA_DEG',
    '3.feed_5975.PM2_5': 'PM2_5_4',
    '3.feed_23.PM10_UG_M3': 'PM10_UG_M3',
    '3.feed_27.NO_PPB': 'NO_PPB',
    '3.feed_27.NOY_PPB': 'NOY_PPB',
    '3.feed_27.CO_PPB': 'CO_PPB',
    '3.feed_27.SO2_PPB': 'SO2_PPB',
    '3.feed_29.PM25_UG_M3..3.feed_29.PM25T_UG_M3': 'PM2_5_5',
    '3.feed_26.PM25B_UG_M3..3.feed_26.PM25T_UG_M3..3.feed_59665.PM25_640_UG_M3': 'PM2_5_6',
}

#### Avec le backdoor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_regression
import copy


debug=False
def get_combination(lst,tuplelst):
    i=0
    new_tuplelst=[]
    if len(tuplelst)==0:
        l=lst[0]
        for v in l:
            new_tuplelst.append([v])
        if len(lst)>1:
            return get_combination(lst[1:],new_tuplelst)
        else:
            return new_tuplelst
    

    currlst=lst[0]
    for l in tuplelst:
        
        for v in currlst:
            newl=copy.deepcopy(l)
            newl.append(v)
            new_tuplelst.append(newl)
        
    if len(lst)>1:
        return get_combination(lst[1:],new_tuplelst)
    else:
        return new_tuplelst
      
def get_C_set(df,C):
    lst=[]
    for Cvar in C:
        lst.append(list(set(list(df[Cvar]))))
    combination_lst= (get_combination(lst,[]))
    return combination_lst

def get_val(row,target,target_val):
    i=0
    while i<len(target):
        if not int(row[target[i]])==int(target_val[i]):
            return 0
        i+=1
    return 1

def train_regression(df,conditional,conditional_values,target,target_val):
    new_lst=[]
    count=0
    for index,row in df.iterrows():
        new_lst.append(get_val(row,target,target_val))
        if new_lst[-1]==1:
            count+=1
    if len(conditional)==0:
        return count*1.0/df.shape[0]
    if len(list(set(new_lst)))==1:
        if new_lst[0]==1:
            return 1
        else:
            return 0
    
    if len(conditional)>0:
        X=df[conditional]
    else:
        X=df
    #print(X)
    #print(new_lst)
    regr = RandomForestRegressor(random_state=0)
    #regr = LogisticRegression(random_state=0)
    regr.fit(X.values, new_lst)
    return regr

def get_prob_o_regression(df,conditional,conditional_values,target,target_val):
    new_lst=[]
    count=0
    for index,row in df.iterrows():
        new_lst.append(get_val(row,target,target_val))
        if new_lst[-1]==1:
            count+=1
    if len(conditional)==0:
        return count*1.0/df.shape[0]
    if len(list(set(new_lst)))==1:
        if new_lst[0]==1:
            return 1
        else:
            return 0
        
    if len(conditional)>0:
        X=df[conditional]
    else:
        X=df
    regr = RandomForestRegressor(random_state=0)
    #regr = LogisticRegression(random_state=0)
    regr.fit(X.values, new_lst)
    #print (regr.coef_.tolist())
    #print (regr.predict_proba([conditional_values]),"ASDFDS")
    return (regr.predict([conditional_values])[0])
    #return(regr.predict_proba([conditional_values])[0][1])

In [ ]:

def get_query_output(df,q_type,AT,prelst,prevallst,postlst,postvallst,Ac,c,g_Ac_lst,interference, blocks):

    #print (len(sub_df),len(sub_intervene))
    if q_type=='count':
        conditioning_set=prelst
        #        intervention=
        backdoorlst=[]

        for attr in Ac:
            backdoorlst.extend(backdoor[attr])
            print('estoy aqui')
        backdoorlst=list(set(backdoorlst))

        if len(backdoorlst)>0:
            backdoorvals=get_C_set(df,backdoorlst)
            #print(backdoorvals)
        else:
            backdoorvals=[]
            
        total_prob=0
        regr=''
        iter=0
        print('hola')
        for backdoorvallst in backdoorvals:
            print('chao')
            conditioning_set=[]
            conditioning_set.extend(prelst)
            #print(conditioning_set)
            conditioning_set.extend(Ac)
            #print(conditioning_set)
            conditioning_set.extend(backdoorlst)
            #print(conditioning_set)

            conditioning_val=[]
            conditioning_val.extend(prevallst)
            #print(conditioning_val)
            conditioning_val.extend(c)
            conditioning_val.extend(backdoorvallst)

            print ("conditioning set",conditioning_set,conditioning_val)
            print("post condition",postlst,postvallst)
            if iter==0:
                regr=train_regression(df,conditioning_set,conditioning_val,postlst,postvallst)
            pogivenck= regr.predict([conditioning_val])[0]#(get_prob_o_regression(df,conditioning_set,conditioning_val,postlst,postvallst))
            
            
            #print(prelst,prevallst,backdoorlst,backdoorvallst)
            pcgivenk = (get_prob_o_regression(df,prelst,prevallst,backdoorlst,backdoorvallst))
            #print (pogivenck,pcgivenk)
            total_prob+=pogivenck * pcgivenk
            iter+=1
            
        #print("final prob is ",total_prob)
        return total_prob

#### Prepare data ZIPCODE 15133 for the training

In [ ]:
df = pd.read_csv('data/merged_data_15133.csv')

df.rename(columns=columns, inplace=True)
df.rename(columns={'smell_1': 'smell'}, inplace=True)
columns_to_evaluate = df.filter(regex='^(?!smell_)').columns.tolist()
df = df.dropna(subset=columns_to_evaluate, how='all')
df.drop(columns=['DateTime','smell_2','smell_3'], inplace=True)

In [ ]:
def convertWindDirection(df):
    df_cp = df.copy(deep=True)
    for c in df.columns:
        if "SONICWD_DEG" in c or "@" in c:
            df_c = df[c]
            df_c.name = df_c.name.replace("@", "")
            df_c_cos = np.cos(np.deg2rad(df_c))
            df_c_sin = np.sin(np.deg2rad(df_c))
            df_c_cos.name += "cosine"
            df_c_sin.name += "sine"
            df_cp.drop([c], axis=1, inplace=True)
            df_cp[df_c_cos.name] = df_c_cos
            df_cp[df_c_sin.name] = df_c_sin
    return df_cp

In [ ]:
# null treatment
df.iloc[:, 1:] = df.iloc[:, 1:].replace(-1.0, np.nan)
df.iloc[:, 1:] = df.iloc[:, 1:].fillna(df.iloc[:, 1:].mean())
df['smell'] = df['smell'].fillna('no smell')

# wind direction conversion
df = convertWindDirection(df)

# standardization
# df_mean = df.iloc[:, 1:].mean()
# df_std = df.iloc[:, 1:].std()
# df.iloc[:, 1:] = (df.iloc[:, 1:] - df_mean) / df_std

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df.iloc[:, 1:] = scaler.fit_transform(df.iloc[:, 1:])


# # smell distribution
# import matplotlib.pyplot as plt
# import seaborn as sns

# smell_counts = df['smell'].value_counts()
# plt.figure(figsize=(10, 6))
# sns.barplot(x=smell_counts.index, y=smell_counts.values)

# plt.title('Distribution of "Smell" in Zipcode 15133')
# plt.xlabel('Smell')
# plt.ylabel('Distribution')
# plt.xticks(rotation=45)  # Rotar las etiquetas del eje x para mejor legibilidad
# plt.tight_layout()

# plt.show()

# one hot encoding
df = pd.get_dummies(df, columns=['smell'])

In [ ]:
df

In [ ]:
backdoor={}
for var in df.columns:
    backdoor[var]=['SONICWD_DEGsine', 'SONICWD_DEGcosine']

print(backdoor)

scores = {}

for col in ['H2S_PPM']:
    values= list(set(df[col].values))
    scores[col]=[]
    for v in values:
        print(v)
        scores[col].append(get_query_output(df,'count','',[],[],['smell_egg'],[1],[col],[v],['*'],'',{}))#,{0:[1,2]}))


In [ ]:
scores

In [ ]:
import json

with open('scores_15133.json', 'w') as f:
    json.dump(scores, f)